In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/My Drive/aniReco/Dataset')
! ls

anime_cleaned.csv	 final_cleaned_model_ready.csv	     UserAnimeList.csv
anime_filtered.csv	 final_cleaned_model_ready_ver2.csv  UserList.csv
AnimeList.csv		 final_cleaned_model_ready_ver3.csv  users_cleaned.csv
animelists_cleaned.csv	 name_id_mapping.pickle		     users_filtered.csv
animelists_filtered.csv  output_dataset


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

%matplotlib inline

In [13]:
orig_anime_df = pd.read_csv('anime_cleaned.csv')
orig_anime_df.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727.0,366,204003,2579,NaN,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508.0,1173,70127,802,NaN,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,False,"Aug 16, 2002 to May 23, 2003","{'from': '2002-08-16', 'to': '2003-05-23'}",16 min. per ep.,PG-13 - Teens 13 or older,8.21,36501,307.0,916,93312,3344,Princess Tutu aired in two parts. The first pa...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,False,"Oct 6, 2012 to Mar 30, 2013","{'from': '2012-10-06', 'to': '2013-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,8.67,107767,50.0,426,182765,2082,NaN,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [5]:
anime_df = pd.read_csv('final_cleaned_model_ready_ver3.csv')
anime_df.head()

,anime_id,type,source,episodes,status,airing,rating,score,scored_by,rank,popularity,members,favorites,studio,duration_min,aired_from_year,Comedy,Supernatural,Romance,Shounen,Parody,School,Magic,Shoujo,Drama,Fantasy,Kids,Action,Music,SliceofLife,Josei,Harem,ShounenAi,Adventure,SuperPower,Sci-Fi,Ecchi,Seinen,MartialArts,Game,Sports,Demons,Historical,Horror,Mystery,Psychological,Vampire,Mecha,Military,Space,Samurai,Thriller,Hentai,Yaoi,ShoujoAi,Police,Cars,Dementia,Yuri,unknown,days
0,11013,5,6,12,1,0,3,7.63,139250,1022.0,216,283882,2809,139,24.0,2012.0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77.0
1,2104,5,6,26,1,0,3,7.89,91206,610.0,332,204003,2579,207,24.0,2007.0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,182.0
2,5262,5,6,51,1,0,2,7.55,37129,1179.0,1017,70127,802,470,24.0,2008.0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,356.0
3,721,5,9,38,1,0,3,8.21,36501,281.0,801,93312,3344,225,16.0,2002.0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,280.0
4,12365,5,6,25,1,0,3,8.67,107767,49.0,388,182765,2082,250,24.0,2012.0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,175.0


## Cosine Similarity

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
anime_features = anime_df.loc[:, "type":].copy()
anime_features.head()

,type,source,episodes,status,airing,rating,score,scored_by,rank,popularity,members,favorites,studio,duration_min,aired_from_year,Comedy,Supernatural,Romance,Shounen,Parody,School,Magic,Shoujo,Drama,Fantasy,Kids,Action,Music,SliceofLife,Josei,Harem,ShounenAi,Adventure,SuperPower,Sci-Fi,Ecchi,Seinen,MartialArts,Game,Sports,Demons,Historical,Horror,Mystery,Psychological,Vampire,Mecha,Military,Space,Samurai,Thriller,Hentai,Yaoi,ShoujoAi,Police,Cars,Dementia,Yuri,unknown,days
0,5,6,12,1,0,3,7.63,139250,1022.0,216,283882,2809,139,24.0,2012.0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77.0
1,5,6,26,1,0,3,7.89,91206,610.0,332,204003,2579,207,24.0,2007.0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,182.0
2,5,6,51,1,0,2,7.55,37129,1179.0,1017,70127,802,470,24.0,2008.0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,356.0
3,5,9,38,1,0,3,8.21,36501,281.0,801,93312,3344,225,16.0,2002.0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,280.0
4,5,6,25,1,0,3,8.67,107767,49.0,388,182765,2082,250,24.0,2012.0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,175.0


In [25]:
anime_df.iloc[5492]

anime_id    16706.0
type            5.0
source          6.0
episodes       12.0
status          1.0
             ...   
Cars            0.0
Dementia        0.0
Yuri            0.0
unknown         0.0
days           77.0
Name: 5492, Length: 61, dtype: float64

In [9]:
cosine_sim = cosine_similarity(anime_features.values, anime_features.values)

In [10]:
cosine_sim

array([[1.        , 0.99935842, 0.99917781, ..., 0.00860203, 0.01001023,
        0.00883641],
       [0.99935842, 1.        , 0.99749965, ..., 0.00960985, 0.01101222,
        0.00984379],
       [0.99917781, 0.99749965, 1.        , ..., 0.02982036, 0.0312036 ,
        0.03012739],
       ...,
       [0.00860203, 0.00960985, 0.02982036, ..., 1.        , 0.99984694,
        0.99951216],
       [0.01001023, 0.01101222, 0.0312036 , ..., 0.99984694, 1.        ,
        0.99975824],
       [0.00883641, 0.00984379, 0.03012739, ..., 0.99951216, 0.99975824,
        1.        ]])

In [11]:
cosine_sim.shape

(6621, 6621)

In [38]:
anime_index = pd.Series(anime_df.index, index=anime_df.anime_id)
anime_index

anime_id
11013       0
2104        1
5262        2
721         3
12365       4
         ... 
37894    6616
37405    6617
37255    6618
35229    6619
36315    6620
Length: 6621, dtype: int64

In [20]:
def get_recommendation(anime_id, similarity=cosine_sim):
    idx = anime_index[anime_id]
    
    # Get the pairwsie similarity scores of all anime with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar anime
    sim_scores = sim_scores[0:11]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar anime
    result = anime_df[['anime_id', 'score']].iloc[anime_indices].drop(idx)
    return result

In [33]:
result = get_recommendation(12365)
anime_ids = result.anime_id.values

In [80]:
show_recc = pd.DataFrame(columns=['anime_id','title','genre','score'])
for id in anime_ids:
    show_recc = show_recc.append(orig_anime_df[['anime_id','title','genre','score']].loc[orig_anime_df.anime_id==id], ignore_index=True)

In [81]:
show_recc.head(10)

,anime_id,title,genre,score
0,16706,Kami nomi zo Shiru Sekai: Megami-hen,"Harem, Comedy, Supernatural, Romance, Shounen",8.22
1,9617,K-On! Movie,"Music, Slice of Life, Comedy",8.34
2,5341,Ookami to Koushinryou II,"Adventure, Historical, Romance, Fantasy",8.43
3,73,Full Metal Panic! The Second Raid,"Action, Military, Mecha",8.03
4,16782,Kotonoha no Niwa,"Slice of Life, Psychological, Drama, Romance",8.27
5,268,Golden Boy,"Adventure, Comedy, Ecchi",8.04
6,11577,Steins;Gate Movie: Fuka Ryouiki no Déjà vu,"Sci-Fi, Drama",8.58
7,5681,Summer Wars,"Sci-Fi, Comedy",8.22
8,1195,Zero no Tsukaima,"Action, Adventure, Harem, Comedy, Magic, Roman...",7.54
9,7338,Darker than Black: Kuro no Keiyakusha Gaiden,"Action, Sci-Fi, Mystery, Super Power",8.05


## KNN 

In [82]:
from sklearn.neighbors import NearestNeighbors

In [83]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(anime_features)

In [84]:
distances, indices = nbrs.kneighbors(anime_features)

In [110]:
indices

array([[   0, 2432, 5922, ..., 4465, 1238, 1570],
       [   1,  870,   89, ..., 6199, 3902,   44],
       [   2, 2380, 1778, ..., 3196,  471, 4629],
       ...,
       [6618, 6582, 3758, ..., 6581, 3093, 5754],
       [6619, 6615, 4283, ..., 6596, 2005, 4719],
       [6620, 4576, 3940, ..., 2355, 6615, 6591]])

In [111]:
distances

array([[    0.        ,  3132.27696772,  5192.16891375, ...,
        11422.87433191, 11725.59185345, 11850.45712205],
       [    0.        ,  2780.5464348 ,  2925.85885116, ...,
         6286.14818649,  6392.27870344,  6649.99633612],
       [    0.        ,  1566.8401361 ,  1798.00977108, ...,
         2209.04665962,  2330.12845657,  2353.22332344],
       ...,
       [    0.        ,    50.18760405,    59.33203182, ...,
          119.22670171,   121.83702229,   128.82676896],
       [    0.        ,    60.90511391,    77.97749675, ...,
          113.84078751,   119.33395535,   126.63352005],
       [    0.        ,    66.30763531,    95.6792893 , ...,
          153.79821878,   171.60897412,   174.82549471]])

In [91]:
indices.shape

(6621, 10)

In [104]:
idx = anime_index[12365]
idx

4

In [122]:
anime_df[['anime_id']].iloc[4].values[0]

12365

In [123]:
def print_similar_animes(anime_id):
    idx = anime_index[anime_id]
    show_recc = pd.DataFrame(columns=['anime_id','title','genre','score'])
    for id in indices[idx][1:]:
        show_recc = show_recc.append(orig_anime_df[['anime_id','title','genre','score']].loc[orig_anime_df.anime_id==anime_df[['anime_id']].iloc[id].values[0]], ignore_index=True)
    return show_recc

In [125]:
print_similar_animes(13667)

,anime_id,title,genre,score
0,5258,Hajime no Ippo: New Challenger,"Comedy, Sports, Drama, Shounen",8.73
1,4437,Naruto: Shippuuden Movie 2 - Kizuna,"Action, Martial Arts, Shounen, Supernatural",7.41
2,4280,Kara no Kyoukai 4: Garan no Dou,"Action, Mystery, Supernatural, Thriller",8.00
3,12729,High School DxD OVA,"Comedy, Demons, Ecchi, Romance, School",7.45
4,32380,Kono Subarashii Sekai ni Shukufuku wo! OVA,"Adventure, Comedy, Supernatural, Magic, Fantasy",7.86
5,585,Mimi wo Sumaseba,"Slice of Life, Drama, Romance, Shoujo",8.30
6,3901,Baccano! Specials,"Action, Mystery, Comedy, Historical, Supernatural",8.24
7,6201,Princess Lover!,"Harem, Comedy, Ecchi, School",6.96
8,597,Neko no Ongaeshi,"Adventure, Drama, Fantasy",7.94
